<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/english_To_French_UsingSEQ2SEQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
!unzip /content/french_eng.zip

Archive:  /content/french_eng.zip
  inflating: fra.txt                 


In [3]:
from __future__ import print_function
from keras.models import Model
from keras.layers import LSTM, Input, Dense

In [4]:
data_path='/content/french_eng.txt'

## DATA PREPROCESSING

In [5]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding='utf-8') as f:
  lines=f.read().split('\n')




In [6]:
lines

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)',
 'Run!\tCours\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)',
 'Run!\tCourez\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)',
 'Who?\tQui ?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #4366796 (gillux)',
 'Wow!\tÇa alors\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #374631 (zmoo)',
 'Fire!\tAu feu !\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #4627939 (sacredceltic)',
 "Help!\tÀ l'aide\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #128430 (sysko)",
 'Jump.\tSaute.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishi

In [7]:
for i in lines[:10000]:
    parts = i.split('\t')

    input, target, _ = parts
    input_texts.append(input)
    target = '\t' + target + '\n'
    target_texts.append(target)
    for char in input:
          if char not in input_characters:
            input_characters.add(char)
    for char in target:
          if char not in target_characters:
            target_characters.add(char)


In [8]:
len(input_texts)

10000

In [9]:
len(target_texts)

10000

In [10]:
target_texts[0]

'\tVa !\n'

In [11]:
input_texts[100]

'Come in.'

In [12]:

target_texts

['\tVa !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\tJe comprends.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tJ’ai gagné.\n',
 '\tOh non !\n',
 '\tAttaque !\n',
 '\tAttaquez !\n',
 '\tSanté !\n',
 '\tÀ votre santé !\n',
 '\tMerci !\n',
 '\tTchin-tchin !\n',
 '\tLève-toi.\n',
 '\tVa, maintenant.\n',
 '\tAllez-y maintenant.\n',
 '\tVas-y maintenant.\n',
 "\tJ'ai pigé !\n",
 '\tCompris !\n',
 '\tPigé\u202f?\n',
 '\tCompris\u202f?\n',
 "\tT'as capté\u202f?\n",
 '\tMonte.\n',
 '\tMontez.\n',
 '\tSerre-moi dans tes bras !\n',
 '\tSerrez-moi dans vos bras !\n',
 '\tJe suis tombée.\n',
 '\tJe suis tombé.\n',
 '\tJe sais.\n',
 '\tJe suis parti.\

In [13]:
input_characters

{' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [14]:
len(input_characters)

70

In [15]:
len(target_characters)

93

In [16]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

In [33]:
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(seq) for seq in input_texts])
max_decoder_seq_length=max([len(seq) for seq in target_texts])

In [18]:
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_len)
print('Max sequence length for outputs:', max_decoder_seq_len)

Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [19]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])

In [20]:
target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])

## CREATING ENCODER AND DECODER

In [21]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_len, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype='float32')

In [22]:
batch_s = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dimension = 256  # Latent dimensionality of the encoding space.
number_samples = 10000  # Number of samples to train on.

In [23]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

## Encoder

zip(input_texts, target_texts) creates an iterator that generates pairs of elements from input_texts and target_texts. For example, if input_texts is ['text1', 'text2', 'text3'] and target_texts is ['target1', 'target2', 'target3'], zip will produce pairs like ('text1', 'target1'), ('text2', 'target2')

In [24]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dimension, return_state=True)    #return dequence is false because we dont need output
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

## Decoder

In [25]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dimension, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

 the dense layer is a critical component in neural networks, providing the network with the capacity to learn complex relationships in the data through the combination of linear transformations and non-linear activation functions.

In neural networks, a dense layer, also known as a fully connected layer, is a fundamental building block that connects each neuron in one layer to every neuron in the next layer. This layer is called "dense" because of the dense connections between the neurons. The dense layer performs a linear operation on its input, followed by an activation function.

In [26]:
#Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/5
125/125 [==============================] - 11s 30ms/step - loss: 1.3596 - accuracy: 0.7165 - val_loss: 1.1175 - val_accuracy: 0.6928
Epoch 2/5
125/125 [==============================] - 2s 14ms/step - loss: 0.9173 - accuracy: 0.7451 - val_loss: 0.9630 - val_accuracy: 0.7401
Epoch 3/5
125/125 [==============================] - 2s 13ms/step - loss: 0.7792 - accuracy: 0.7894 - val_loss: 0.8208 - val_accuracy: 0.7738
Epoch 4/5
125/125 [==============================] - 1s 12ms/step - loss: 0.6674 - accuracy: 0.8116 - val_loss: 0.7282 - val_accuracy: 0.7899
Epoch 5/5
125/125 [==============================] - 1s 12ms/step - loss: 0.6090 - accuracy: 0.8272 - val_loss: 1.3324 - val_accuracy: 0.7537


In [29]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [30]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [31]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [34]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Go.
Decoded sentence: Estez !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Hi.
Decoded sentence: Elle te !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Hi.
Decoded sentence: Elle te !

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Run!
Decoded sentence: Artez !

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Run!
Decoded sentence: Artez !

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Who?
Decoded sentence: Nous ait ?

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Wow!
Decoded sentence: Nous en  a                                                  
1/1 [==============================] - 0s 20ms/step
-
Input sentence: Fire!
Decoded sentence: Sois en                                                     
1/1 [==============================] - 0s 18ms/step
-
Input sentence: Help!

KeyboardInterrupt: ignored